/usr/lib/jvm/java-11-openjdk-amd64/bin/java -cp /home/selim/PENG9630/bgp_data_collector/peng_env/lib/python3.7/site-packages/pyspark/conf:/home/selim/PENG9630/bgp_data_collector/peng_env/lib/python3.7/site-packages/pyspark/jars/* -Xmx24g org.apache.spark.deploy.SparkSubmit --conf spark.master=local[16] --conf spark.driver.memory=24g --conf spark.kryoserializer.buffer.max=1g --conf spark.driver.maxResultSize=2g --conf spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.app.name=PENG research pyspark-shell

### Track the extent and evolution of the use of BGP communities.
### To this end, please use a set of routing tables from BGPStream that covers the period 2011-2021.
### More specifically, use all the routing tables that were available in the first seven days of January of every second year, that is 2011, 2013, 2015, 2017, 2019 and 2021.
### Then, identify how many ASes are using communities, rank ASes based on the number of prefixes that are tagged with communities, check what tier-1 ASes use communities for by matching the used communities to what they list publicly.

you can use RIBs.
In terms of data sources please use RRC00, RRC01, RRC03,RRC04, RRC05

I just shared with you a document that describes three course projects.
Please pick one of them by the end of tomorrow.
The results are supposed to be submitted in the form of a mini report and the deadline is June 27th.
On June 28th, we will have a meeting where everyone will go orally through his/her report.
Please let me know if you have questions or comments.
One more thing, in light of the current tight timeline, we will skip the last paper presentation and just focus on the project.
I would also like to draw your attention to the fact that these projects are by no means weekend projects.
Depending on your background, you may need to dedicate between two and three weeks.

https://labs.ripe.net/author/florian_streibelt/bgp-communities-a-weapon-for-the-internet-part-1/


sources:  
https://www.inet.tu-berlin.de/fileadmin/fg234_teaching/SS14/IM_SS14/im14_05_measuring_networks.pdf  
http://bgpresearch.blogspot.com/2013/10/tools_29.html  
https://docs.hydrolix.io/tutorials/bgp  
https://ahermosilla.com/tools/2020/07/07/where-ip-data-lives.html  


Q: "The main thing I need to figure out is what part of the RIB entry can be used to find out where an advertisement originated (as in, which AS originated an advertisement), though I still want to understand what all the other fields mean (both for RIB and Update)."

A: The format is a little confusing, but the originating AS is actually the right most AS number in the AS_PATH field. Thus, the originating AS is 2906. Although unlikely, it should be noted that there are no security mechanisms to verify BGP announcements and any AS along the path can modify the announcement.

(Adding numbers to different parts of the AS Path to explain better):

ASPATH: 41695{3} 174{2} 174 3356 2906{1}

1: This is the originating AS

2: Here you can see something called "AS Path Prepending". The BGP protocol is a path cost based protocol. Thus, here in this example some AS along the path chose to represent this prefix as having a cost of 5 instead of what we humans can see is a cost of 4 if you don't duplicate paths. There are many reasons AS operators might do this and I won't describe the reasons here.

3: This is the neighboring AS

A few other fields I'll explain:

"PREFIX: 198.45.50.0/24"

This is really the meat of BGP. This is the IP address(es) being advertised by the announcement. I would read this BGP announcement in its simplest form as "AS 2906 wants to advertise the prefix of 198.45.50.0/24. If you are next to AS 41695, then your traffic will have to traverse the AS path of (read left to right now) 41695 174 174 3356 2906 to get to this IP range".

"FROM: 195.66.226.74 AS41695"

This is the AS and the IP address (also known as 'peer') that sent the announcement to whatever vantage point you have. Thus the 'From' announcement can be read as "AS41695 is passing along this prefix announcement from it's IP of 195.66.226.74"

COMMUNITY: 174:21000 174:22013 41695:1000 41695:1003 41695:1103

The BGP communities field basically allows AS operators finer grained control in routing traffic and determining policy. It's a 32 bit field split by the ":" character. The values are agreed upon between two ASes so it's hard to know what each one means if the AS doesn't publicly disclose what it means. As a general rule the first 16 bits represent the AS that makes the announcement and the second 16 bits represent some signal or action. Thus we can read "174:21000" as "AS 174 request action '21000' to happen for this prefix" or "AS 174 tags this prefix with signal '21000'" (whatever action/signal it is). Some ASes do in fact publish this info. AT&T does.

In general I highly suggest you read the RFCs such as RFC 4271. Although they can be dense depending on your level of understanding, they have most of the information to understand everything. https://tools.ietf.org/html/rfc4271#section-5.1

  

rib|R|1438416001.000000|ris|rrc06|None|None|25152|202.249.2.185|185.84.166.0/23|202.249.2.185|25152 2914 39386 29684 41054|2914:410 2914:1405 2914:2406 2914:3400 2914:3475|None|None  
rib|R|1438416001.000000|ris|rrc06|None|None|25152|202.249.2.185|185.84.166.0/24|202.249.2.185|25152 2914 39386 29684 41054|2914:410 2914:1405 2914:2406 2914:3400 2914:3475|None|None  
rib|R|1438416001.000000|ris|rrc06|None|None|25152|202.249.2.185|185.84.167.0/24|202.249.2.185|25152 2914 39386 29684 41054|2914:410 2914:1405 2914:2406 2914:3400 2914:3475|None|None  
Community: 2914:410 ==> 185.84.166.0/24,185.84.167.0/24,185.84.166.0/23  
Community: 2914:1405 ==> 185.84.166.0/24,185.84.167.0/24,185.84.166.0/23  
Community: 2914:2406 ==> 185.84.166.0/24,185.84.167.0/24,185.84.166.0/23  
Community: 2914:3400 ==> 185.84.166.0/24,185.84.167.0/24,185.84.166.0/23  
Community: 2914:3475 ==> 185.84.166.0/24,185.84.167.0/24,185.84.166.0/23  

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import concat, col, lit
from pyspark import StorageLevel
from datetime import datetime
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


In [3]:
spark = SparkSession \
    .builder \
    .master("local[40]") \
    .appName("PENG research") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config('spark.driver.memory','120g') \
    .config('spark.driver.maxResultSize', '2g') \
    .config('spark.kryoserializer.buffer.max', '1g') \
    .config('spark.local.dir', '/global/D1/homes/selim/spark-tmp-dir/') \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [4]:
def my_udf(val1):
    return datetime.utcfromtimestamp(int(val1)).strftime('%Y')

udf_timestamp = udf(my_udf, StringType())

In [5]:
columns = ['type', 'timestamp', 'dump_type_W/A/B(withdrawal/announcement/routing table)', 'Peer_IP(address_of_the_monitor)', 'Peer_ASN(ASN_of_the_monitor)', 'announced_prefix', 'ASPath', 'Origin_Protocol', 'Next_Hop', 'LocalPref', 'MED', 'Community_strings', 'Atomic_Aggregator', 'Aggregator', 'IX']
# sample_paths = [
# '/global/D1/homes/selim/sample_dataset/rrc00_bview.20110101.1600_bgpdump',
# '/global/D1/homes/selim/sample_dataset/rrc01_bview.20130101.0000_bgpdump',
# '/global/D1/homes/selim/sample_dataset/rrc05_bview.20170101.1600_bgpdump'
# ]

result_directory = '/global/D1/homes/selim/result_dir_rrc05/'

# paths = [
#     '/global/D1/homes/selim/rrc_00_ribs_data_bgpdump/*_bgpdump',
#     '/global/D1/homes/selim/rrc_01_ribs_data_bgpdump/*_bgpdump',
#     '/global/D1/homes/selim/rrc_03_ribs_data_bgpdump/*_bgpdump',
#     '/global/D1/homes/selim/rrc_04_ribs_data_bgpdump/*_bgpdump',
#     '/global/D1/homes/selim/rrc_05_ribs_data_bgpdump/*_bgpdump'
# ]

home_paths = [
    # '/home/selim/rrc_00_ribs_data_bgpdump/*_bgpdump',
    # '/global/D1/homes/selim/rrc_01_ribs_data_bgpdump/*_bgpdump',
    # '/global/D1/homes/selim/rrc_03_ribs_data_bgpdump/*_bgpdump',
    # '/home/selim/rrc_04_ribs_data_bgpdump/*_bgpdump',
    '/global/D1/homes/selim/rrc_05_ribs_data_bgpdump/*_bgpdump'
]

bgpdump_df = spark.read.csv(path=home_paths, sep='|').toDF(*columns).filter(col('Community_strings').isNotNull()).filter(col('announced_prefix').isNotNull()).withColumn("date", udf_timestamp(col('timestamp'))).persist(storageLevel=StorageLevel.MEMORY_ONLY)
bgpdump_df.show(5, truncate=False)

+-----------+----------+------------------------------------------------------+-------------------------------+----------------------------+----------------+--------------+---------------+-------------+---------+---+----------------------------------------------------------------------------------------------------+-----------------+----------+----+----+
|type       |timestamp |dump_type_W/A/B(withdrawal/announcement/routing table)|Peer_IP(address_of_the_monitor)|Peer_ASN(ASN_of_the_monitor)|announced_prefix|ASPath        |Origin_Protocol|Next_Hop     |LocalPref|MED|Community_strings                                                                                   |Atomic_Aggregator|Aggregator|IX  |date|
+-----------+----------+------------------------------------------------------+-------------------------------+----------------------------+----------------+--------------+---------------+-------------+---------+---+----------------------------------------------------------------------

## TASK 1: identify how many ASes are using communities  

In [5]:
community_df = bgpdump_df.select('date', 'Community_strings')
community_rdd = community_df.rdd
community_df.show(5, truncate=False)

+----+----------------------------------------------------------------------------------------------------+
|date|Community_strings                                                                                   |
+----+----------------------------------------------------------------------------------------------------+
|2011|286:18 286:19 286:28 286:29 286:800 286:888 286:3049 286:4017 3549:4025 3549:30840 4788:200 4788:210|
|2011|1120:1 no-export                                                                                    |
|2011|3303:1004 3303:1009 3303:3058                                                                       |
|2011|1853:1119 1853:1120 4788:200 4788:210                                                               |
|2011|1120:2 4788:200 4788:210 8596:40330                                                                 |
+----+----------------------------------------------------------------------------------------------------+
only showing top 5 rows



In [6]:
def community_split_map_function(row):
    return_values = []
    for asn_community in row.Community_strings.split(' '):
        return_values.append(row.date + '_' + asn_community.split(':')[0])
    return return_values


community_string ayristiriliyor. her bir asn aliniyor. date_asn formatinda unique degerler aliniyor.  
alinan unique asn degerleri yillara gore sayiliyor

In [7]:
community_rdd.map(community_split_map_function ).flatMap(lambda x: x).take(15)

['2011_286',
 '2011_286',
 '2011_286',
 '2011_286',
 '2011_286',
 '2011_286',
 '2011_286',
 '2011_286',
 '2011_3549',
 '2011_3549',
 '2011_4788',
 '2011_4788',
 '2011_1120',
 '2011_no-export',
 '2011_3303']

In [8]:
community_rdd.map(community_split_map_function ).flatMap(lambda x: x).distinct().map(lambda x: (x.split('_')[0], 1)  ).reduceByKey(lambda x, y: x + y).repartition(1).saveAsTextFile(path=result_directory + 'task_1_asn_count')


## TASK 2: rank ASes based on the number of prefixes that are tagged with communities  
bu asn kac tane prefix ile eslenmis|

In [6]:
rank_asn_df = bgpdump_df.select('date', 'Community_strings', 'announced_prefix')
rank_asn_rdd = rank_asn_df.rdd
rank_asn_df.show(7, truncate=False)

+----+----------------------------------------------------------------------------------------------------+----------------+
|date|Community_strings                                                                                   |announced_prefix|
+----+----------------------------------------------------------------------------------------------------+----------------+
|2011|286:18 286:19 286:28 286:29 286:800 286:888 286:3049 286:4017 3549:4025 3549:30840 4788:200 4788:210|1.9.0.0/16      |
|2011|1120:1 no-export                                                                                    |1.9.0.0/16      |
|2011|3303:1004 3303:1009 3303:3058                                                                       |1.9.0.0/16      |
|2011|1853:1119 1853:1120 4788:200 4788:210                                                               |1.9.0.0/16      |
|2011|1120:2 4788:200 4788:210 8596:40330                                                                 |1.9.0.0/16      |


### Task 2, first way, eliminating community type, prefixes are taken into the account,  
as numbers with same prefixes are ranked here

In [10]:
def rank_map_function(row):
    date = row.date
    community_array = row.Community_strings.split(' ')
    prefix = row.announced_prefix
    result_list = []

    for community_string in community_array:
        community_string = community_string.split(':')[0]
        key = date + '_' + community_string + '_' + prefix
        result_list.append((key, 1))
    return result_list


ASN x is the most used with specific prefix

In [11]:
rank_asn_rdd.map(rank_map_function).flatMap(lambda x: x).take(15)

[('2011_286_1.9.0.0/16', 1),
 ('2011_286_1.9.0.0/16', 1),
 ('2011_286_1.9.0.0/16', 1),
 ('2011_286_1.9.0.0/16', 1),
 ('2011_286_1.9.0.0/16', 1),
 ('2011_286_1.9.0.0/16', 1),
 ('2011_286_1.9.0.0/16', 1),
 ('2011_286_1.9.0.0/16', 1),
 ('2011_3549_1.9.0.0/16', 1),
 ('2011_3549_1.9.0.0/16', 1),
 ('2011_4788_1.9.0.0/16', 1),
 ('2011_4788_1.9.0.0/16', 1),
 ('2011_1120_1.9.0.0/16', 1),
 ('2011_no-export_1.9.0.0/16', 1),
 ('2011_3303_1.9.0.0/16', 1)]

In [12]:
rank_asn_rdd.map(rank_map_function).flatMap(lambda x: x).reduceByKey(lambda x, y: x + y).sortBy(keyfunc=lambda x: x[1], ascending=False).repartition(1).saveAsTextFile(path=result_directory + 'task_2_1st_way')


### task 2, second way, prefix included, community type included,  
prefix and community type are take into the account  


In [13]:
def rank_map_function_1(row):
    date = row.date
    community_array = row.Community_strings.split(' ')
    prefix = row.announced_prefix
    result_list = []

    for community_string in community_array:
        #community_string = community_string.split(':')[0]
        key = date + '_' + community_string + '_' + prefix
        result_list.append((key, 1))
    return result_list


ASN x is the most used one with community type t and a specific prefix

In [14]:
rank_asn_rdd.map(rank_map_function_1).flatMap(lambda x: x).take(15)

[('2011_286:18_1.9.0.0/16', 1),
 ('2011_286:19_1.9.0.0/16', 1),
 ('2011_286:28_1.9.0.0/16', 1),
 ('2011_286:29_1.9.0.0/16', 1),
 ('2011_286:800_1.9.0.0/16', 1),
 ('2011_286:888_1.9.0.0/16', 1),
 ('2011_286:3049_1.9.0.0/16', 1),
 ('2011_286:4017_1.9.0.0/16', 1),
 ('2011_3549:4025_1.9.0.0/16', 1),
 ('2011_3549:30840_1.9.0.0/16', 1),
 ('2011_4788:200_1.9.0.0/16', 1),
 ('2011_4788:210_1.9.0.0/16', 1),
 ('2011_1120:1_1.9.0.0/16', 1),
 ('2011_no-export_1.9.0.0/16', 1),
 ('2011_3303:1004_1.9.0.0/16', 1)]

In [15]:
rank_asn_rdd.map(rank_map_function_1).flatMap(lambda x: x).reduceByKey(lambda x, y: x + y).sortBy(keyfunc=lambda x: x[1], ascending=False).repartition(1).saveAsTextFile(path=result_directory + 'task_2_2nd_way')

### task 2, another way, counting all asn numbers, includes more than one asn numbers with same prefixes

In [7]:
def rank_map_function_2(row):
    result_list = []

    for community_string in row.Community_strings.split(' '):
        key = row.date + '_' + community_string.split(':')[0]
        result_list.append((key, 1))
    return result_list


ASN x is the most used one, includes same prefixes too.  
This is based on as numbers, one asn can have same prefix counts and other different prefix counts also.

In [8]:
rank_asn_rdd.map(rank_map_function_2).flatMap(lambda x: x).take(15)

[('2011_286', 1),
 ('2011_286', 1),
 ('2011_286', 1),
 ('2011_286', 1),
 ('2011_286', 1),
 ('2011_286', 1),
 ('2011_286', 1),
 ('2011_286', 1),
 ('2011_3549', 1),
 ('2011_3549', 1),
 ('2011_4788', 1),
 ('2011_4788', 1),
 ('2011_1120', 1),
 ('2011_no-export', 1),
 ('2011_3303', 1)]

In [18]:
rank_asn_rdd.map(rank_map_function_2).flatMap(lambda x: x).reduceByKey(lambda x, y: x + y).sortBy(keyfunc=lambda x: x[1], ascending=False).repartition(1).saveAsTextFile(path=result_directory + 'task_2_3rd_way')

### task 2, another way, counting all asn numbers with unique prefixes

In [19]:
def rank_map_function_3(row):
    date = row.date
    community_array = row.Community_strings.split(' ')
    prefix = row.announced_prefix
    result_list = []

    for community_string in community_array:
        community_string = community_string.split(':')[0]
        key = date + '_' + community_string + '_' + prefix
        result_list.append(key)
    return result_list

def prefix_split_func(row):
    row_array = row.split('_')
    return row_array[0] + '_' + row_array[1]

ASN x is the most used one, only unique prefixes. Same date_asn_prefix values are counted as 1.  


In [20]:
rank_asn_rdd.map(rank_map_function_3).flatMap(lambda x: x).take(15)

['2011_2914_1.9.0.0/16',
 '2011_2914_1.9.0.0/16',
 '2011_2914_1.9.0.0/16',
 '2011_2914_1.9.0.0/16',
 '2011_35054_1.9.0.0/16',
 '2011_3549_1.9.0.0/16',
 '2011_3549_1.9.0.0/16',
 '2011_4788_1.9.0.0/16',
 '2011_4788_1.9.0.0/16',
 '2011_25091_1.9.0.0/16',
 '2011_25091_1.9.0.0/16',
 '2011_2914_1.11.128.0/17',
 '2011_2914_1.11.128.0/17',
 '2011_2914_1.11.128.0/17',
 '2011_2914_1.11.128.0/17']

In [21]:
rank_asn_rdd.map(rank_map_function_3).flatMap(lambda x: x).distinct().map(prefix_split_func).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).sortBy(keyfunc=lambda x: x[1], ascending=False).repartition(1).saveAsTextFile(path=result_directory + 'task_2_4th_way')

## TASK 3: check what tier-1 ASes use communities for by matching the used communities to what they list publicly

List of Tier 1 networks   https://en.wikipedia.org/wiki/Tier_1_network  
These networks are universally recognized as Tier 1 networks, because they can reach the entire internet (IPv4 and IPv6) via settlement-free peering. The CAIDA AS rank is a rank of importance on the internet.[10]  
  
| Name                                                                                            | Headquarters       | AS number | CAIDA AS rank[10] | Fiber route (km) | Peering policy                                          |
|-------------------------------------------------------------------------------------------------|--------------------|-----------|-------------------|------------------|---------------------------------------------------------|
| AT&T[11]                                                                                        | United States      | 7018      | 21                | 660,000[12]      | AT&T Peering policy                                     |
| Deutsche Telekom Global Carrier[13]                                                             | Germany            | 3320      | 20                | 250,000[14]      | DTAG Peering Details                                    |
| GTT Communications                                                                              | United States      | 3257      | 3                 | 232,934[15][16]  | GTT Peering Policy                                      |
| Liberty Global[17][18]                                                                          | United Kingdom[19] | 6830      | 24                | 800,000[20]      | Peering Principles                                      |
| Lumen Technologies (formerly CenturyLink formerly Level 3)[21][22][23]                          | United States      | 3356      | 1                 | 885,139[24][25]  | CenturyLink Peering Policy; Level 3 Peering Policy      |
| Lumen Technologies (formerly CenturyLink formerly Level 3 formerly Global Crossing)[21][22][23] | United States      | 3549      | 10                | 885,139[24][25]  | CenturyLink Peering Policy; Level 3 Peering Policy      |
| NTT Communications (formerly Verio)[26]                                                         | Japan              | 2914      | 5                 | ?                | Global Peering Policy                                   |
| Orange (OpenTransit)[27]                                                                        | France             | 5511      | 14                | ?                | OTI peering policy                                      |
| PCCW Global                                                                                     | Hong Kong          | 3491      | 11                | ?                | Peering policy                                          |
| Sprint (T-Mobile US)[28]                                                                        | United States      | 1239      | 32                | 42,000[29]       | Peering policy                                          |
| Tata Communications (formerly Teleglobe)[30]                                                    | India              | 6453      | 8                 | 700,000[31]      | Peering Policy                                          |
| Telecom Italia Sparkle (Seabone)[32]                                                            | Italy              | 6762      | 6                 | 560,000          | Peering Policy                                          |
| Telia Carrier[33]                                                                               | Sweden             | 1299      | 2                 | 65,000[34]       | TeliaSonera International Carrier Global Peering Policy |
| Telxius (Subsidiary of Telefónica)[35]                                                          | Spain              | 12956     | 16                | 65,000[36]       | Peering Policy                                          |
| Verizon Enterprise Solutions (formerly UUNET)[41]                                               | United States      | 701       | 19                | 805,000[42]      | Verizon UUNET Peering policy 701, 702, 703              |
| Zayo Group (formerly AboveNet)[43]                                                              | United States      | 6461      | 9                 | 196,339[44]      | Zayo Peering Policy                                     |


In [22]:
#tier_1_asn_list = [7018, 3320, 3257, 6830, 3356, 3549, 2914, 5511, 3491, 1239, 6453, 6762, 1299, 12956, 701, 6461]

In [23]:
tier_1_df = spark.read.csv(path='./tier-1-networks.csv', header=True).withColumn('name_hq', concat(col('Name'), lit('-'), col('Headquarters'))).drop('Name', 'Headquarters')
tier_1_df.show(5, truncate=False)

+---------+---------------------------------------+
|AS_number|name_hq                                |
+---------+---------------------------------------+
|7018     |AT&T-United States                     |
|3320     |Deutsche Telekom Global Carrier-Germany|
|3257     |GTT Communications-United States       |
|6830     |Liberty Global-United Kingdom          |
|3356     |Lumen Technologies-United States       |
+---------+---------------------------------------+
only showing top 5 rows



In [24]:
community_df_task3 = bgpdump_df.select('date', 'Community_strings')
community_rdd_t3 = community_df_task3.rdd
community_df_task3.show(4, truncate=False)

+----+--------------------------------------------------------------------+
|date|Community_strings                                                   |
+----+--------------------------------------------------------------------+
|2011|2914:410 2914:1402 2914:2403 2914:3400 35054:103                    |
|2011|3549:4025 3549:30840 4788:200 4788:210 25091:10 25091:3549          |
|2011|2914:410 2914:1005 2914:2000 2914:3000 2914:3075 2914:3275 35054:103|
|2011|13030:1 13030:1009 13030:51701 25091:10 25091:13030                 |
+----+--------------------------------------------------------------------+
only showing top 4 rows



In [25]:
def community_split_map_function_t3(row):
    date = row.date
    community_string_array = row.Community_strings.split(' ')
    return_values = []
    for asn_community in community_string_array:
        asn_community_array = asn_community.split(':')
        if len(asn_community_array) == 2:
            return_values.append(Row(date=date, AS_number=asn_community_array[0], community_type=asn_community_array[1]))
    return return_values


In [26]:
asn_df = community_rdd_t3.map(community_split_map_function_t3).flatMap(lambda x: x).toDF()

In [27]:
tier_1_community_df_1 = asn_df.join(other=tier_1_df, on='AS_number', how='inner').distinct()

In [28]:
tier_1_community_df_2 = tier_1_community_df_1.withColumn('key', concat(col('date'), lit('_'), col('AS_number'), lit('_'), col('name_hq'))).drop('AS_number', 'name_hq', 'date').select('key', 'community_type')

In [29]:
#tier_1_community_df_2.show(20, truncate=False)

In [30]:
#tier_1_community_df_1.show(20, truncate=False)

In [31]:
tier_1_community_df_3 = tier_1_community_df_2.rdd.groupByKey().mapValues(list)


In [32]:
def key_splitter_function(row):
    key = row[0]
    date, asn, organization = key.split('_')
    return Row(date=date, asn=asn, organization=organization, community_type_length=row[1])

In [33]:
tier_1_community_df_4 = tier_1_community_df_3.mapValues(len).map(key_splitter_function).toDF()

In [34]:
tier_1_community_df_4.orderBy(['date', 'community_type_length'], ascending=[True, False]).repartition(1).write.csv(path=result_directory + 'task_3', header=True)